In [1]:
import pandas as pd
import numpy as np
import json

#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets\credits.csv', dtype='unicode')

pd.set_option("display.max_rows", 2000, "display.max_columns", None)

In [2]:
#TRANSFORMACIONES VARIAS ---> NO OLVIDAR CORRER LO QUE ESTA COMENTADO

#lleno NaNs con 0s en las columnas revenue y budget
#elimino rows que contienen un NaN en la columna release_date
#elimino columnas que no se utilizaran
try:
    dfMovies['revenue'] = dfMovies['revenue'].fillna(0)
    dfMovies['budget'] = dfMovies['budget'].fillna(0)
    dfMovies = dfMovies.dropna(subset='release_date')
    dfMovies= dfMovies.drop(columns=["video","adult","imdb_id" ,"original_title", "poster_path", "homepage"])
except:
    print("estas transformaciones ya se ejecutaron")


In [3]:
#FUNCIONES

#desanida columna y devuelve un dataframe
def desanidarColumna(columna):
    lista = []
    listaDict=[]
    counter=0
    for k, v in columna.items():
        if isinstance(v, str):
            lista.append(v)
        else:
            lista.append(None)
    for i in lista:   
                try:
                    i=json.loads(lista[counter].replace("'", '"'))
                    listaDict.append(i)
                    counter+=1
                except:
                    listaDict.append(None)
                    counter+=1
    listaDict=pd.json_normalize(listaDict)
    return listaDict

def desanidarVariasColumnas(columna):
    lista=[]
    for col in columna:            
        #normalizo cada columna
        colAnidada=pd.json_normalize(columna[col])
        lista.append(colAnidada) #meto cada columna en una lista

        #armo un df concatenando cada columna de la lista
        nuevoDF=pd.concat(lista)
    return nuevoDF


In [4]:
#desanido belongs to collection

Títulos = (desanidarColumna(dfMovies['belongs_to_collection']))
#dfMovies["Títulos"] = Títulos["name"] es otra forma de agregar la columna
#agrego la columna al inicio

#inserto nueva columna y elimino la vieja
dfMovies.insert(0, "Títulos", Títulos["name"])
dfMovies= dfMovies.drop(columns=["belongs_to_collection"])

In [5]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion

dfGeneros = (desanidarColumna(dfMovies['genres'])) #le aplico la primera funcion
dfGeneros=desanidarVariasColumnas(dfGeneros) #desanido cada columna con la segunda funcion      
dfGeneros=dfGeneros.iloc[:45378] #esto es cualquier cosa pero bueno, funciona xd
                                 #uso iloc para obtener solo hasta el row 45378 porque sino tengo errores
                                 
#inserto nueva columna y elimino la vieja
dfMovies.insert(1, "Géneros", dfGeneros["name"])
dfMovies= dfMovies.drop(columns=["genres"])

In [6]:
#desanido tabla production_companies
dfCompanies=desanidarColumna(dfMovies['production_companies'])
dfCompanies=desanidarVariasColumnas(dfCompanies)
dfCompanies= dfCompanies.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(7, "Compañías", dfCompanies["name"])
dfMovies= dfMovies.drop(columns=["production_companies"])

In [7]:
#desanido production_countries
dfProdCountries=desanidarVariasColumnas(desanidarColumna(dfMovies["production_countries"]))
dfProdCountries = dfProdCountries.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(8, "Países", dfProdCountries["name"])
dfMovies= dfMovies.drop(columns=["production_countries"])

In [16]:
#desanido spoken_languages
dfSpokenLan=desanidarVariasColumnas(desanidarColumna(dfMovies["spoken_languages"]))
dfSpokenLan = dfSpokenLan.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(12, "Lenguaje", dfSpokenLan["name"]) #puedo elegir entre poner la abreviacion o la palabra entera
dfMovies= dfMovies.drop(columns=["spoken_languages"])

,Títulos,Géneros,budget,id,original_language,overview,popularity,Compañías,Países,release_date,revenue,runtime,Lenguaje,status,tagline,title,vote_average,vote_count
0,Toy Story Collection,Animation,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033,81.0,English,Released,NaN,Toy Story,7.7,5415
1,NaN,Adventure,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,262797249,104.0,English,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413
2,Grumpy Old Men Collection,Romance,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.,United States of America,1995-12-22,0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92
3,NaN,Comedy,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34
4,Father of the Bride Collection,Comedy,0,11862,en,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions,United States of America,1995-02-10,76578911,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,NaN,NaN,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,NaN,NaN,1991-05-13,0,104.0,NaN,Released,NaN,Robin Hood,5.7,26
45462,NaN,NaN,0,111109,tl,An artist struggles to finish his work while a...,0.178241,NaN,NaN,2011-11-17,0,360.0,NaN,Released,NaN,Century of Birthing,9.0,3
45463,NaN,NaN,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,NaN,NaN,2003-08-01,0,90.0,NaN,Released,A deadly game of wits.,Betrayal,3.8,6
45464,NaN,NaN,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,NaN,NaN,1917-10-21,0,87.0,NaN,Released,NaN,Satan Triumphant,0.0,0


In [ ]:
#tablas que contienen fechas:
    #release_date
release_year = dfMovies["release_date"]
#cambio valores de fechas invalidas a fechas default
release_year[19730] = "1900-01-01"
release_year[29503] = "1900-01-01"
release_year[35587] = "1900-01-01"

#obtengo solo año de cada string de fecha y lo meto en release_year
dfMovies["release_date"] = pd.to_datetime(release_year).dt.year
dfMovies

#se supone que transforme la columna en date type pero es un int REVISAR


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506
